In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-04-18 01:56:18--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.69MB/s    in 0.2s    

2021-04-18 01:56:18 (5.69 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
#df.describe().show()


In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":'count'}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42596797|             1|
|   12296525|             1|
|   49101505|             1|
|   28377689|             1|
|   14468845|             1|
|   26079415|             1|
|   12945150|             1|
|   42049189|             1|
|     740134|             1|
|   15123832|             1|
|   43742600|             2|
|     134205|             2|
|     255890|             1|
|   52748596|             2|
|   30070208|             1|
|   20994440|             1|
|   45330946|             1|
|   18061621|             1|
|    1051742|             1|
|   38273165|             1|
+-----------+--------------+
only showing top 20 rows



In [25]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id','product_title']).drop_duplicates(subset=['product_id'])

products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0000732QL|   Eastpak Unplugged|
|B00015GEIQ|AmeriLeather Leat...|
|B0001N6F00|Western Pack Off ...|
|B0001ZTM3U|Royce Leather Aut...|
|B000AM1BQA|LaSelva Designs M...|
|B000AYJN2M|U.S. Luggage Busi...|
|B000BV069I|Victorinox CS2-Cr...|
|B000CS7PEY|Ladies Red Hot Ba...|
|B000E99YS6|Personalized Leat...|
|B000HUAXRI|Clava Leather Upr...|
|B000LZ121A|Kenneth Cole Reac...|
|B000MNB93M|Travel Accessorie...|
|B000OTKR7S|Case Logic DVD Al...|
|B000PWIZJQ|Rick Steves Veloc...|
|B000SKLK6K|Alice Pack Frame ...|
|B000UO7ZFO|Travelers Club Lu...|
|B000VIMO7I|AmeriLeather Leat...|
|B000XT5WY2|WATERSHED Yukon D...|
|B000Y4WO38|   Jansport Shoe Bag|
|B000YV66N0|Military Style 2 ...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id',"customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R9CO86UUJCAW5|   40884699|B00VGTN02Y|     786681372| 2015-08-31|
|R3PR8X6QGVJ8B1|   23208852|B005KIWL0E|     618251799| 2015-08-31|
|R39BO2819ABUPF|   17100246|B007UNSHJ6|     810480328| 2015-08-31|
|R3ADL7V6EGGEEP|   13533670|B00WHFNXB4|      20765193| 2015-08-31|
|R1OXYPBPLVRMI5|   38541771|B000M5NBYU|     812890709| 2015-08-31|
|R1WYM8Z5ATQ98O|   12686499|B00ACBJ89G|     624906499| 2015-08-31|
|R3LCIANTN1H9EC|   27767206|B001H4BQ1A|     385044506| 2015-08-31|
|R3U2M23N1P0KQ6|   46630606|B00ESG6NDK|      51297652| 2015-08-31|
|R29MB6N7HB6NZI|   41622754|B00M14SAXC|     832113872| 2015-08-31|
| RGEQ6DGRG7DQG|   15296380|B012PC5QAY|     490793867| 2015-08-31|
|R38HNH0BSS2KFE|   21206415|B00VWKWWMG|     356411837| 2015-08-31|
|R2CATRM9CH59LY|   34554097|B00N2U14UK|     326835930| 2015-08

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9CO86UUJCAW5|          3|            0|          0|   N|                Y|
|R3PR8X6QGVJ8B1|          5|            0|          0|   N|                Y|
|R39BO2819ABUPF|          4|            0|          0|   N|                Y|
|R3ADL7V6EGGEEP|          4|            0|          0|   N|                Y|
|R1OXYPBPLVRMI5|          5|            0|          0|   N|                Y|
|R1WYM8Z5ATQ98O|          3|            0|          0|   N|                Y|
|R3LCIANTN1H9EC|          4|            1|          1|   N|                Y|
|R3U2M23N1P0KQ6|          5|            0|          0|   N|                Y|
|R29MB6N7HB6NZI|          1|            2|          2|   N|                Y|
| RGEQ6DGRG7DQG|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [14]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.csumm5fimgpt.us-east-2.rds.amazonaws.com:5432/amazon_vine" 
config = {"user":"postgres", 
          "password": "database-1", 
          "driver":"org.postgresql.Driver"}

In [16]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [26]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [18]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [27]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)